In [ ]:
## smtlib3
# https://smtlib.cs.uiowa.edu/version3.shtml

# Obviously biting off too big a piece

In [ ]:


import lark

grammar = """

command : 
  | declareconst
  | declaredatatype
  | open
  | import
  | assert
  | definemodule

  

expr : 
    int
   | "(" expr expr* ")"
   | "(" "!" "Type" )
   | "(" "lambda" "(" symbol sort ")" expr ")"
   | "(" "let" "(" "(" symbol expr ")" ")" expr ")"
   | "(" "forall" "(" "(" symbol sort ")" ")" expr ")"
   | "(" "exists" "(" "(" symbol sort ")" ")" expr ")"
   | "(" "match" expr "(" "(" symbol symbol ")" expr ")" ")"
   | "(" "choose" "(" symbol sort ")" expr ")"

   
basetype : "Int" | "Bool" | "BitVec"
type : 
      | "(" "->" type* type ")"
      | "(" "Pi" "(" symbol sort ")" type ")"
      | "(" "Forall" "(" symbol sort ")" type ")"


defineconst : "(" "define-const" symbol sort ")"
symbol : /[a-zA-Z_][a-zA-Z0-9_]*/
sort : symbol
"""

In [10]:
#https://gist.github.com/pib/240957
from string import whitespace

atom_end = set('()"\'') | set(whitespace)

def parse(sexp):
    stack, i, length = [[]], 0, len(sexp)
    while i < length:
        c = sexp[i]

        print(c, stack)
        reading = type(stack[-1])
        if reading == list:
            if   c == '(': stack.append([])
            elif c == ')': 
                stack[-2].append(stack.pop())
                if stack[-1][0] == ('quote',): stack[-2].append(stack.pop())
            elif c == '"': stack.append('')
            elif c == "'": stack.append([('quote',)])
            elif c in whitespace: pass
            else: stack.append((c,))
        elif reading == str:
            if   c == '"': 
                stack[-2].append(stack.pop())
                if stack[-1][0] == ('quote',): stack[-2].append(stack.pop())
            elif c == '\\': 
                i += 1
                stack[-1] += sexp[i]
            else: stack[-1] += c
        elif reading == tuple:
            if c in atom_end:
                atom = stack.pop()
                if atom[0][0].isdigit(): stack[-1].append(eval(atom[0]))
                else: stack[-1].append(atom)
                if stack[-1][0] == ('quote',): stack[-2].append(stack.pop())
                continue
            else: stack[-1] = ((stack[-1][0] + c),)
        i += 1
    return stack.pop()

In [ ]:
def sexp(toks):
    if toks[0] == "(":
        i = sexp(toks[1:])
        toks[0] = ")"
        return i+1
    

In [11]:
parse("(define-const x Int)")

( [[]]
d [[], []]
e [[], [], ('d',)]
f [[], [], ('de',)]
i [[], [], ('def',)]
n [[], [], ('defi',)]
e [[], [], ('defin',)]
- [[], [], ('define',)]
c [[], [], ('define-',)]
o [[], [], ('define-c',)]
n [[], [], ('define-co',)]
s [[], [], ('define-con',)]
t [[], [], ('define-cons',)]
  [[], [], ('define-const',)]
  [[], [('define-const',)]]
x [[], [('define-const',)]]
  [[], [('define-const',)], ('x',)]
  [[], [('define-const',), ('x',)]]
I [[], [('define-const',), ('x',)]]
n [[], [('define-const',), ('x',)], ('I',)]
t [[], [('define-const',), ('x',)], ('In',)]
) [[], [('define-const',), ('x',)], ('Int',)]
) [[], [('define-const',), ('x',), ('Int',)]]


[[('define-const',), ('x',), ('Int',)]]

In [25]:
import re

# \s* is whitespace follow by digits or symbol or ( or )
token_pattern = re.compile(r"\s*(?:(\d+)|([A-Za-z\-!=\+\*\_<>]+[A-Za-z0-9\-!=\+\*\_<>]*)|(\()|(\)))")

def tokenize(s):
    for match in token_pattern.finditer(s):
        yield match.groups()

def parse_expression(iterator):
    """Parse an expression from the token iterator."""
    items = []
    for number, symbol, lparen, rparen in iterator:
        if lparen:
            items.append(parse_expression(iterator))
        elif rparen:
            return items
        elif number:
            items.append(int(number))
        elif symbol:
            items.append(symbol)
        else:
            raise SyntaxError("Unexpected token")
    return items

def parse_sexp(s):
    """Parse an S-expression from a string."""
    tokens = tokenize(s)
    try:
        # The outermost list is not required for a valid S-expression,
        # so we extract the single expression inside it.
        result = parse_expression(tokens)
        # Check for trailing tokens
        for _ in tokens:
            raise SyntaxError("Trailing tokens")
        return result
    except StopIteration:
        raise SyntaxError("Unexpected end of input")

# Example usage
s_expression = "(define square (lambda (x) (* x x)))"
parsed = parse_sexp(s_expression)
print(parsed)

# possibly I could make this a generator that streams?
parse_sexp("1 2 4 (foo bar) (8 9 (biz baz))")
parse_sexp("(define-const x!8 Int)")
parse_sexp("""
           (define-const x Int)
           (assert (= x 3))
           (assert (>= x 8))
           (check-sat)
           (push)
           (pop)

           ()
           """)


[['define', 'square', ['lambda', ['x'], ['*', 'x', 'x']]]]


[['define-const', 'x', 'Int'],
 ['assert', ['=', 'x', 3]],
 ['assert', ['>=', 'x', 8]],
 ['check-sat'],
 ['push'],
 ['pop'],
 []]